In [65]:
import matplotlib.pyplot as plt
import pylab as py
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.stats as sm_stats
import statsmodels.stats.api as sms
import scipy.stats as stats
from sklearn import preprocessing
from numpy.random import seed
from numpy.random import rand
from numpy.random import randn
from numpy import mean
from numpy import var
from math import sqrt
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [54]:
personal_data = pd.read_csv("Dataset/personal_train.csv")
other_data = pd.read_csv("Dataset/other_train.csv")

In [55]:
if 'Unnamed: 0' in personal_data:
    del personal_data['Unnamed: 0']
if 'Unnamed: 0' in other_data:
    del other_data['Unnamed: 0']

In [58]:
unique_medical_name_dataset = other_data.dropna(subset=['medical_info']).drop_duplicates('name')
unique_medical_name_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3933 entries, 0 to 3982
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             3933 non-null   object 
 1   address          3933 non-null   object 
 2   kurtosis_oxygen  3929 non-null   float64
 3   occupation       3926 non-null   object 
 4   marital-status   3925 non-null   object 
 5   pregnant         3929 non-null   object 
 6   education-num    3535 non-null   float64
 7   relationship     3926 non-null   object 
 8   std_oxygen       3925 non-null   float64
 9   capital-gain     3928 non-null   float64
 10  skewness_oxygen  3925 non-null   float64
 11  education        3926 non-null   object 
 12  fnlwgt           3928 non-null   float64
 13  class            3923 non-null   float64
 14  income           3927 non-null   object 
 15  medical_info     3933 non-null   object 
 16  native-country   3931 non-null   object 
 17  capital-loss  

In [59]:
# create a dataset from 'medical_info' attribute
medical_data_objects = []
for index, record in unique_medical_name_dataset.iterrows():
    if isinstance(record['medical_info'], float):
        continue
    medical_object = json.loads(record['medical_info'].replace("\'", '\"').replace(':\"',':').replace('\",',',').replace('\"}','}'))
    medical_object['name'] = record['name']
    medical_data_objects.append(medical_object)
medical_info_dataset = pd.DataFrame(medical_data_objects)
medical_info_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933 entries, 0 to 3932
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mean_glucose      3933 non-null   float64
 1   std_glucose       3933 non-null   float64
 2   kurtosis_glucose  3933 non-null   float64
 3   skewness_glucose  3933 non-null   float64
 4   name              3933 non-null   object 
dtypes: float64(4), object(1)
memory usage: 153.8+ KB


In [60]:
# merge datasets to create single large dataset with usefull data so it's easier to create graphs and analysis
merged_medical_info_dataset = unique_medical_name_dataset.merge(medical_info_dataset, on=['name'], how='outer').drop('medical_info', axis='columns')
usefull_dataset = personal_data.merge(merged_medical_info_dataset, on=['name', 'address'], how='outer')
usefull_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3933 entries, 0 to 3932
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              3933 non-null   object 
 1   address           3933 non-null   object 
 2   age               3933 non-null   int64  
 3   sex               3933 non-null   object 
 4   date_of_birth     3933 non-null   object 
 5   kurtosis_oxygen   3929 non-null   float64
 6   occupation        3926 non-null   object 
 7   marital-status    3925 non-null   object 
 8   pregnant          3929 non-null   object 
 9   education-num     3535 non-null   float64
 10  relationship      3926 non-null   object 
 11  std_oxygen        3925 non-null   float64
 12  capital-gain      3928 non-null   float64
 13  skewness_oxygen   3925 non-null   float64
 14  education         3926 non-null   object 
 15  fnlwgt            3928 non-null   float64
 16  class             3923 non-null   float64
